In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. 讀入需要的套件

這裡主要用 `LangChain`, 這可以說整合各式 LLM 功能的方便套件。

In [3]:
!pip install -q pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 8.1 MB/s eta 0:00:00


In [4]:
!pip install -U nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.9.1
    Uninstalling nltk-3.9.1:
      Successfully uninstalled nltk-3.9.1


In [5]:
!pip install langchain langchain-community openai faiss-cpu unstructured tiktoken langchain-google-genai

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

讀入正確的 `nltk` 所需資料。

In [1]:
import nltk

In [3]:
nltk.data.path.append("/root/nltk_data")
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

讀入一大票需要的函式。

In [4]:
import os
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

### 2. 讀入範例資料

這邊使用林長鋆老師Youtube上所放的<資料科學與迴歸分析講義>，著作權仍屬於老師所有，這邊只是拿來當範例使用。

In [5]:
# 上傳 zip 檔案(手動)

# 解壓縮 books.zip 到 books 資料夾
!7z x "資料科學與回歸分析講義.7z" -obooks


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 48469541 bytes (47 MiB)

Extracting archive: 資料科學與回歸分析講義.7z
--
Path = 資料科學與回歸分析講義.7z
Type = 7z
Physical Size = 48469541
Headers Size = 2074
Method = LZMA:23
Solid = +
Blocks = 1

  0%      3%      7% 2 - 資料科學與回歸分析講義/【 .  迴歸實例（實作）.pdf                                                                               10% 2 - 資料科學與回歸分析講義/【 .  迴歸實例（實作）.pdf                                                                              

### 3. 設定gemini api 金鑰

In [6]:
# 1. 安裝專用套件
#!pip install -q -U langchain-google-genai

# 2. 程式碼範例
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

# 設定 LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=userdata.get('GOOGLE_API_KEY'),
    temperature=0
)

# 測試
result = llm.invoke("你好")
print(result.content)

你好！有什么我可以帮助你的吗？


### 4. 建立向量資料庫

#### Step 1: 加載資料夾中的文件

In [7]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
loader = DirectoryLoader("/content/books/資料科學與回歸分析講義", glob="*.pdf",loader_cls=PyPDFLoader)  # 替換為你的資料夾路徑
documents = loader.load()

In [8]:
print({len(documents)})

{436}


#### Step 2: 將文件分割成較小的片段

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

#### Step 3: 使用 Gemini 的嵌入來將文件轉為向量嵌入

In [10]:
# 初始化 Gemini 嵌入模型
# 使用 models/embedding-001 這個專門的嵌入模型
from google.colab import userdata
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=userdata.get('GOOGLE_API_KEY'))

In [11]:
# 檢查 Step 1 載入的原始文件數量
print(f"載入的原始文件總數 (documents): {len(documents)}")

# 檢查 Step 2 分割後的區塊總數
print(f"分割後的文本區塊總數 (split_docs): {len(split_docs)}")

載入的原始文件總數 (documents): 436
分割後的文本區塊總數 (split_docs): 453


## 暫時使用本地 llm測試

In [12]:
!pip install -q sentence-transformers

In [13]:
# Step 3: 使用本地 HuggingFace 模型來將文件轉換為向量

from langchain_community.embeddings import HuggingFaceEmbeddings

# 使用一個常用的輕量級模型，它會被下載到 Colab 中
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

print("✅ 嵌入模型已切換為本地 HuggingFace 模型。")

/tmp/ipython-input-3213277844.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ 嵌入模型已切換為本地 HuggingFace 模型。


#### Step 4: 使用 FAISS 建立向量資料庫

In [14]:
vector_store = FAISS.from_documents(split_docs, embeddings)

#### Step 5: 建立檢索器

In [15]:
retriever = vector_store.as_retriever()

### 5. 打造分類器機器人

#### 選定語言模型

In [16]:
# 確保匯入正確的類別
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

# 初始化 Gemini Chat Model
# 使用 gemini-2.5-flash 作為目前穩定且快速的模型
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=userdata.get('GOOGLE_API_KEY'), # 傳遞金鑰
    temperature=0.1 # 建議 RAG 查詢的 temperature 不要太高
)

#### 定義一些分類器的回答

In [17]:
# 替換為分類器 (SVM, 隨機森林, 線性模型) 的關鍵定義與回答指令

classification_key_terms = [
    # --- 模型核心概念 ---
    "SVM (支持向量機)：核心概念是尋找一個最大邊界 (Maximum Margin) 的超平面 (Hyperplane) 來分隔不同類別的資料點。關鍵在於選擇合適的核函數 (Kernel Function)。",
    "隨機森林 (Random Forest)：一種集成學習 (Ensemble Learning) 方法，透過建立多棵決策樹 (Decision Trees) 並取多數決 (Voting) 來進行分類，有效減少過擬合 (Overfitting)。",
    "線性模型 (分類)：通常指的是邏輯迴歸 (Logistic Regression)，它使用 Sigmoid 函數將線性預測轉換為機率值，常用於二元分類，並定義決策邊界。",

    # --- 評估與優化 ---
    "評估指標：分類器的性能主要透過混淆矩陣 (Confusion Matrix)、準確率 (Accuracy)、精確率 (Precision)、召回率 (Recall) 和 F1 Score 來評估。",
    "超參數調優 (Hyperparameter Tuning)：例如 SVM 的 C 參數和 Kernel 類型，或隨機森林的樹木數量，這些需要通過交叉驗證 (Cross-Validation) 進行優化。",

    # --- 回答指導原則 ---
    "當回答關於分類器的問題時，請先解釋該模型的**核心原理**與**關鍵參數**，並比較不同模型在處理**非線性資料**或**數據量大**時的優勢與劣勢。",
]

#### 建立一個結合檢索與生成的 RAG 問答鏈

In [18]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

#### 定義真正的心靈處方籤主函式

注意最主要還是設計 `prompt` 的型式。

In [19]:
from langchain.schema import HumanMessage
# 由於不再是隨機抽取心靈處方籤，我們將所有的技術術語合併為指導原則。
# 確保 classification_key_terms 變數已在前面的儲存格中定義。
technical_guidance = "\n".join(classification_key_terms)


def answer_user_question(question):
    # 1. 檢索資料夾中的相關內容
    # 此步驟不變，用於從向量庫中提取與問題相關的文件區塊
    retriever_result = qa_chain.run(question)

    # 2. 自訂 Prompt，結合技術指導原則、上下文和使用者問題
    prompt = f"""
    你的任務是扮演一位專業的機器學習專家，根據以下技術指導原則和檢索到的文件內容來回答問題。

    請嚴格遵循以下技術指導原則：
    {technical_guidance}

    ---
    以下是我們從資料庫中檢索到的內容，這些內容來自書中的資料，並與使用者的問題相關：
    {retriever_result}
    ---

    請根據上述所有資料和指導原則，以專業、嚴謹的語氣，清晰地回應使用者的問題：
    「{question}」
    """

    # 3. 使用 HumanMessage 包裝 prompt 並生成回答
    #    (由於我們將所有指令都放在 Prompt 內，所以繼續使用 HumanMessage)
    final_response = llm.invoke([HumanMessage(content=prompt)])

    return final_response.content

# 備註：由於我們不再隨機抽取，原有的 print(f"你抽到的心靈處方籤: {chosen_prescription}") 語句已移除。

In [20]:
# 使用範例：回答一個機器學習問題 (請確保問題與您的 PDF 文件內容相關)
user_question = "如果面對一個非線性資料集，SVM 中的核函數 (Kernel Function) 是什麼？它如何幫助解決這個問題？"
response = answer_user_question(user_question)

print(f'\n經過機器人得到的內容是 \n==================== \n{response}')

/tmp/ipython-input-2671748350.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever_result = qa_chain.run(question)



經過機器人得到的內容是 
面對非線性資料集時，支持向量機 (SVM) 中的核函數 (Kernel Function) 扮演著至關重要的角色。以下將詳細解釋其核心原理與如何解決非線性問題。

### SVM 的核心原理與非線性資料的挑戰

SVM 的核心原理是尋找一個最大邊界 (Maximum Margin) 的超平面 (Hyperplane) 來分隔不同類別的資料點。對於線性可分的資料，這是一個直接的任務。然而，在許多實際應用中，資料點在原始特徵空間中往往是非線性可分的，這意味著無法透過一個簡單的直線或平面將它們有效區分開來。

### 核函數 (Kernel Function) 是什麼？

核函數是一種數學函數，它允許 SVM 在不顯式地將資料點映射到高維特徵空間的情況下，計算這些點在高維空間中的內積（或相似度）。這個巧妙的技術被稱為「核技巧 (Kernel Trick)」。

具體來說，核函數的目的是：
1.  **隱式映射：** 它將原始低維特徵空間中的資料點，隱式地轉換或「投影」到一個更高維度的特徵空間。
2.  **計算相似度：** 在這個高維空間中，核函數能夠計算兩個資料點之間的相似度（即內積），而無需實際執行複雜的維度轉換。

### 核函數如何幫助解決非線性資料問題？

核函數透過以下機制解決非線性資料集的問題：

1.  **提升維度以實現線性可分性：** 當資料在原始低維空間中非線性可分時，透過核函數將其映射到一個更高維度的空間後，這些資料點在高維空間中可能變得線性可分。想像一下，在二維平面上無法用直線分開的點，在三維空間中可能可以用一個平面輕鬆分開。
2.  **在高維空間中尋找超平面：** 一旦資料在高維空間中變得線性可分，SVM 就可以在這個新的空間中找到一個最大邊界的線性超平面。
3.  **對應原始空間的非線性決策邊界：** 這個在高維空間中找到的線性超平面，當它被「投影」回原始的低維特特徵空間時，就形成了一個複雜的、非線性的決策邊界。這樣，SVM 就能夠有效地對原始的非線性資料進行分類。

### 常見的核函數類型

根據您提供的資料，SVM 中常見的核函數有以下幾種：

*   **Linear (線性核):** $K_{\mathbf{a},\mathbf{b}} = \mathbf{a} + \mathbf{b}$
    *

In [21]:
# 確保您已經在 Step 4 成功建立了 vector_store 變數
# 將向量庫儲存到名為 'faiss_index' 的資料夾中
vector_store.save_local("faiss_index")

print("FAISS 向量庫已成功儲存到 /content/faiss_index 資料夾！")

FAISS 向量庫已成功儲存到 /content/faiss_index 資料夾！
